# Huấn luyện Nhận diện Cảm xúc với Dataset FER-2013
Notebook này huấn luyện mô hình CNN để nhận diện cảm xúc khuôn mặt sử dụng dataset FER-2013 đã tải về local.

**Nguồn Dataset**: https://www.kaggle.com/datasets/msambare/fer2013

In [ ]:
# Import các thư viện cần thiết
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# Khám phá cấu trúc dataset
data_dir = './dataset/emotion'
print("Cấu trúc dataset:")
for root, dirs, files in os.walk(data_dir):
    level = root.replace(data_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files[:3]:  # Chỉ hiển thị 3 file đầu tiên
        print(f"{subindent}{file}")
    if len(files) > 3:
        print(f"{subindent}... và {len(files) - 3} file khác")

In [ ]:
# Định nghĩa các lớp cảm xúc (sử dụng tất cả 7 cảm xúc từ dataset FER-2013)
emotion_classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
emotion_labels = ['Tức giận', 'Ghê tởm', 'Sợ hãi', 'Vui vẻ', 'Bình thường', 'Buồn bã', 'Ngạc nhiên']
num_classes = len(emotion_classes)

print(f"Huấn luyện với {num_classes} lớp cảm xúc: {emotion_labels}")
print("Sử dụng dataset FER-2013 đầy đủ với 7 loại cảm xúc")

In [ ]:
# Tiền xử lý và tăng cường dữ liệu (giảm augmentation để training nhanh hơn)
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=10,  # Giảm từ 20 xuống 10
    width_shift_range=0.1,  # Giảm từ 0.2 xuống 0.1
    height_shift_range=0.1,  # Giảm từ 0.2 xuống 0.1
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generator cho dữ liệu validation (chỉ rescaling)
validation_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
# Tải dữ liệu từ thư mục (tự động phát hiện các lớp cảm xúc)
# Tăng batch_size và giảm kích thước ảnh để training nhanh hơn
train_generator = train_datagen.flow_from_directory(
    directory="./dataset/emotion/train",
    target_size=(64, 64),  # Giảm từ 150x150 xuống 64x64
    batch_size=64,  # Tăng từ 32 lên 64
    class_mode='categorical',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    directory="./dataset/emotion/test",
    target_size=(64, 64),  # Giảm từ 150x150 xuống 64x64
    batch_size=64,  # Tăng từ 32 lên 64
    class_mode='categorical',
    shuffle=False
)

# In thông tin các lớp được phát hiện
detected_classes = list(train_generator.class_indices.keys())
print(f"Các lớp cảm xúc được phát hiện: {detected_classes}")
print(f"Số lượng mẫu training: {train_generator.n}")
print(f"Số lượng mẫu validation: {validation_generator.n}")

# Cập nhật số lượng lớp dựa trên dữ liệu thực tế
num_classes = len(detected_classes)
print(f"Số lượng lớp đã cập nhật: {num_classes}")

In [ ]:
# Xây dựng mô hình CNN cho nhận diện cảm xúc (tối ưu hóa cho tốc độ)
model = Sequential()

# Các lớp Convolutional (giảm số lớp để training nhanh hơn)
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Flatten
model.add(Flatten())

# Các lớp fully-connected
model.add(Dense(64, activation='relu'))  # Giảm từ 128 xuống 64
model.add(Dropout(0.3))  # Giảm dropout từ 0.5 xuống 0.3

# Lớp output
model.add(Dense(num_classes, activation='softmax'))

# Hiển thị kiến trúc mô hình
model.summary()

In [ ]:
# Compile mô hình với optimizer Adam (thường nhanh hơn RMSprop)
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks để tối ưu hóa training
early_stopping = EarlyStopping(
    patience=3, 
    monitor='val_loss', 
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    min_lr=0.0001
)

In [ ]:
# Huấn luyện mô hình (giảm epochs để training nhanh hơn)
print("Bắt đầu huấn luyện mô hình...")
history = model.fit(
    train_generator,
    steps_per_epoch=min(100, train_generator.n // train_generator.batch_size),  # Giới hạn steps
    epochs=10,  # Giảm từ 20 xuống 10
    validation_data=validation_generator,
    validation_steps=min(50, validation_generator.n // validation_generator.batch_size),  # Giới hạn validation steps
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)
print("Hoàn thành huấn luyện!")

In [ ]:
# Đánh giá mô hình
score = model.evaluate(validation_generator, verbose=0)
print(f'Loss trên tập test: {score[0]:.4f}')
print(f'Độ chính xác trên tập test: {score[1]:.4f}')

In [ ]:
# Vẽ biểu đồ lịch sử huấn luyện
plt.figure(figsize=(12, 4))

# Vẽ biểu đồ độ chính xác
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Độ chính xác Training')
plt.plot(history.history['val_accuracy'], label='Độ chính xác Validation')
plt.title('Độ chính xác của Mô hình')
plt.ylabel('Độ chính xác')
plt.xlabel('Epoch')
plt.legend()

# Vẽ biểu đồ loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], 'r', label='Loss Training')
plt.plot(history.history['val_loss'], label='Loss Validation')
plt.title('Loss của Mô hình')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Lưu mô hình đã huấn luyện
model.save('Emotion1.h5')
print("Mô hình đã được lưu với tên 'Emotion1.h5'")

In [ ]:
# Kiểm tra mô hình với ảnh mẫu
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

# Tải mô hình đã lưu
model_CNN = load_model('Emotion1.h5')

# Định nghĩa nhãn dự đoán (tự động phát hiện từ dataset)
# Lưu ý: Thứ tự phải khớp với class indices khi training
try:
    # Thử tải class indices từ file đã lưu hoặc sử dụng mặc định
    predict_labels = ['Tức giận', 'Ghê tởm', 'Sợ hãi', 'Vui vẻ', 'Bình thường', 'Buồn bã', 'Ngạc nhiên']
    print(f"Sử dụng nhãn dự đoán: {predict_labels}")
except:
    predict_labels = ['Cảm_xúc_0', 'Cảm_xúc_1', 'Cảm_xúc_2', 'Cảm_xúc_3', 'Cảm_xúc_4', 'Cảm_xúc_5', 'Cảm_xúc_6']
    print(f"Sử dụng nhãn mặc định: {predict_labels}")

# Hàm dự đoán cảm xúc từ đường dẫn ảnh
def predict_emotion(image_path):
    try:
        # Tải và tiền xử lý ảnh
        img = load_img(image_path, target_size=(64, 64))  # Cập nhật kích thước
        plt.figure(figsize=(6, 6))
        plt.imshow(img)
        plt.title('Ảnh đầu vào')
        plt.axis('off')
        plt.show()
        
        # Chuyển đổi thành array và chuẩn hóa
        img_array = img_to_array(img)
        img_array = img_array.reshape(1, 64, 64, 3)  # Cập nhật kích thước
        img_array = img_array.astype('float32') / 255.0
        
        # Thực hiện dự đoán
        prediction = model_CNN.predict(img_array)
        predicted_class = np.argmax(prediction, axis=-1)[0]
        confidence = np.max(prediction)
        
        print(f"Cảm xúc dự đoán: {predict_labels[predicted_class]}")
        print(f"Độ tin cậy: {confidence:.2%}")
        
        return predict_labels[predicted_class], confidence
    except Exception as e:
        print(f"Lỗi khi xử lý ảnh: {e}")
        return None, None

# Ví dụ sử dụng (bỏ comment và cung cấp đường dẫn ảnh hợp lệ)
# predict_emotion('./dataset/emotion/test/happy/sample_image.jpg')